In [ ]:
%pip install transformers

import pandas as pd
import json
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import ast
import os
import json

model_name = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map= {"":"mps"}
)
print(f"Model device: {model.device}")


Produces an initial prediction for each entity/relation pair.

the examples are from the validation dataset and either based on the relqation type or llm prediction of most similar triples to the target entity/relation.

In [ ]:

with open('./data/val.jsonl', 'r') as file:
    vallist = list(file)

examples = []
for v in vallist:
    r = json.loads(v)
    examples.append((r['SubjectEntity'], r['Relation'], r["ObjectEntities"]))

exampledf = pd.DataFrame(examples, columns = ['SubjectEntity', 'Relation', "ObjectEntities" ] )
rd = exampledf.groupby('Relation').apply(lambda x: x.values.tolist()).to_dict()
cleaneddict = {}

for rel, vals in rd.items():
    textver = ""
    i = 0
    for v in vals:
        if i == 3:
           break
        t = ""
        #

        try:
            t = t + "Q: " + v[0] + " # " + v[1] + "\n" + "A: " + v[2][0]
            for vs in v[2][1:]:
                t = t + " # " + vs
        except:
            t = t + "Q: " + v[0] + " # " + v[1] + "\n" + "A: none"
        textver = textver + t + "\n"
        i = i +1
    cleaneddict[rel] = textver
print(cleaneddict)

examplefile = pd.read_csv("data/testexamples.tsv", sep = "\t")
customexamples = examplefile.set_index('triple')['examples'].to_dict()
print(examplefile)
print(customexamples)

def examples_fetch(entity, rel):
	return customexamples[entity+","+rel]


#takes the entity relation pair and produces an initial 10x set of triples from it
def initial_triple(input, infer_technique, custom = False):
	multianswers = ["companyTradesAtStockExchange", "awardWonBy","countryBordersCountry"]
	newTriples = []
	pInfo = "State facts about {entity}."
	winfo = {}
	i = 0
	for t in input:
		entity = t[0]
		rel = t[1]
		prompt = pInfo.format(entity = entity)
		winfo[i] = {"entity": entity, "relation": rel, "prompt": prompt}
		i = i+1

	keys = list(winfo.keys())
	for i in range(0, len(keys), 5):
		k = keys[i:i + 5]
		cp = [winfo[key]["prompt"] for key in k]

		res = call_llm(cp, "temp_sampling")

		for key, response in zip(k, res):
			winfo[key]["info"] = "C: " + response


	for key, vdict in winfo.items():
		entity = vdict["entity"]
		rel = vdict["relation"]
		info = vdict["info"]

		print(entity, rel, info)

		examples = []
		if custom == True:
			examples = customexamples[entity+","+rel]#custom_examples(entity, rel)
		else:
			examples = cleaneddict[rel]

		print(examples)

		promptTriple  = """{info}\n""" + examples + """\nQ: {entity} # {rel}\nA:"""

		prompt = promptTriple.format(entity=entity, rel=rel, info = info )

		if rel in multianswers:
			gen = 6
		else:
			gen = 5


		#print(prompt)


		res = []
		for _ in range(gen):
			r = call_llm(prompt, infer_technique, batches= 3) #can't do 10 batches, 5 also crashes??
			#print("llm gives" , r)
			#print(len(r))
			for x in r:
				res.append(x)

				#print(x)

		ents = [entity] * len(res)
		rels = [rel] * len(res)

		#print(res)
		#print(ents)
		cleanres =  llm_cleanup(res, ents, rels) # ideally gives list of cleaned stuff

		if cleanres:
			for r in cleanres:
				newTriples.append([entity, rel, r ])
				print([entity, rel, r ])
		else:
			newTriples.append([entity, rel, "none" ])
			print([entity, rel, "none" ])

	df = pd.DataFrame(newTriples, columns=['entity', 'relation', 'object'])
	return df


Re-runs the initial triples, if the llm judge determines them to be bad, re-runs to get a new tail prediction up to 3x times

Each prediction deemed 'incorrect' by the llm judge is included in the re-run prompt in the next round

The good triples and any incorrect ones remaining are both retained and filtered later

In [ ]:
with open('./data/val.jsonl', 'r') as json_file:
    json_list = list(json_file)

examples = []
for json_str in json_list:
    r = json.loads(json_str)
    examples.append((r['SubjectEntity'], r['Relation'], r["ObjectEntities"]))
print(len(examples))
print(examples)
exampledf = pd.DataFrame(examples, columns = ['SubjectEntity', 'Relation', "ObjectEntities" ] )
rd = exampledf.groupby('Relation').apply(lambda x: x.values.tolist()).to_dict()
cleaneddict = {}

for rel, vals in rd.items():
    textver = ""

    for v in vals:
        textver = textver + "Correct triple: " + v[0] + " " + v[1]

        for vs in v[2]:
            textver = textver + " # " + vs

    cleaneddict[rel] = textver
print(cleaneddict)

def faster_check(triples, infer_ver, multi = False):
    goodTriples = []
    withprompt = {}
    promptInfo = "State facts about {entity}."
    prompt = """{info}
    {prev}
    CORRECT: {entity} {rel} # """

    for index, triple in triples.iterrows():
        triple = (triple['entity'], triple['relation'], triple['object'] )

        if 3 > len(triple):
                triple = (triple[0], triple[1], "none")
        try:
            if triple[2] == [] or "answer" in triple[2]:
                triple = (triple[0], triple[1], "none")
        except:
            print("bad triple ", triple)

        withprompt[index] = {"triple": triple, "prompt": promptInfo.format(entity = triple[0]), "prevTriples" : [], "judge" : "no"} #setting default judge bc it kept crashing

    #so should send through and get all the info at once so only do it once in all
    keys = list(withprompt.keys())
    for i in range(0, len(keys), 5):
        k = keys[i:i + 5]
        cp = [withprompt[key]["prompt"] for key in k]

        res = call_llm(cp, "temp_sampling")

        for key, response in zip(k, res):
            withprompt[key]["info"] = response


    for runum in range(5):


        updatedkeys = list(withprompt.keys())
        if not updatedkeys:
            break #if it runs out before finishing 5 sets
#JUDGE
        for i in range(0, len(updatedkeys), 5):
            k = updatedkeys[i:i + 5]
            cp = [withprompt[key]["triple"] for key in k]
            try:
                res = minijudge(cp, single = True)
                for key, response in zip(k, res):
                    #print(response)
                    withprompt[key]["judge"] = response
            except:
                for key in k: #if it crashes, just send it back next time
                    print("judge went weird")
                    withprompt[key]["judge"] = "no"

    #TAKE OUT GOOD ONES
        goodones = []
        for key, item_dict in withprompt.items():
            #print(item_dict)
            if "yes" in item_dict["judge"]:
                goodones.append(key)
                goodTriples.append(item_dict["triple"])
            else:
                #withprompt[key]["prevTriples"] =  withprompt[key]["prevTriples"].append(item_dict["triple"])
                withprompt[key]["prevTriples"].append(item_dict["triple"])
                cleanedPrev = []

                #print(item_dict["prevTriples"])
                for tup in item_dict["prevTriples"]:
                    #print(tup, prevtriples)
                    cleanedPrev.append("INCORRECT: " + tup[0] + " " + tup[1] + " # " + str(tup[2]))

                prev = '\n'.join(cleanedPrev)
                withprompt[key]["prompt"] = prompt.format(prev = prev, entity = item_dict["triple"][0], rel = item_dict["triple"][1], info = item_dict["info"])


        for key in goodones:
            del withprompt[key]

        rerunkeys = list(withprompt.keys())

    #BATCH RE-RUN BAD ONES

        for i in range(0, len(rerunkeys), 5):
            k = rerunkeys[i:i + 5]
            cp = [withprompt[key]["prompt"] for key in k]
            ents = [withprompt[key]["triple"][0] for key in k]
            rels = [withprompt[key]["triple"][1] for key in k]

            res = call_llm(cp, "temp_sampling")


            newTails = llm_cleanup(res, ents, rels)

            for key, response in zip(k, newTails):

                #withprompt[key]["triple"] = (triple[0], triple[1], response)
                ogtriple = withprompt[key]["triple"]
                withprompt[key]["triple"] = (ogtriple[0], ogtriple[1], response)

    #SHOULD WORK TO DO THIS 3x, the bad ones will straight up just get dropped though so ideally run this a lot
        df = pd.DataFrame(goodTriples, columns = ["entity", "rel", "tail"])
        df.to_csv("data/" + str(runum) + ".tsv", sep = "\t")

    #adding in the not great ones just in case, they'll probably get filtered out anyway
    for key, item_dict in withprompt.items():
      goodTriples.append(item_dict["triple"])

    return goodTriples






The call to the llm, used temp_sampling for the generation

In [ ]:

def call_llm(prompts, infer_technique, more_tokens=False, batches=1, t=.15):
    if isinstance(prompts, str):
        #print("single prompt")
        batch_list = [prompts] * batches
    else:
        batch_list = prompts
        batches = len(prompts)

    inputs = tokenizer(batch_list, return_tensors="pt", padding=True, truncation=True).to(model.device)

    if more_tokens:
        mnt = 50
    else:
        mnt = 15

    gen_kwargs = {
        "max_new_tokens": mnt,
        "attention_mask": inputs.attention_mask,
        "pad_token_id": tokenizer.eos_token_id
    }

    if infer_technique == "greedy":
        with torch.no_grad():
            generated_ids = model.generate(inputs.input_ids, **gen_kwargs)
    elif infer_technique == "qwenrec":
        gen_kwargs.update({
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.8,
            "top_k": 20,
            "stop_strings": ["\n", "Q: "],
            "tokenizer": tokenizer
        })
        with torch.no_grad():
            generated_ids = model.generate(inputs.input_ids, **gen_kwargs)
    elif infer_technique == "temp_sampling":
        gen_kwargs.update({
            "do_sample": True,
            "top_k": 0,
            "temperature": t,
            "stop_strings": ["\n", "Q: "],
            "tokenizer": tokenizer
        })
        with torch.no_grad():
            generated_ids = model.generate(inputs.input_ids, **gen_kwargs)
    else:
        return "generation messed up"

    output_ids = []
    for i in range(generated_ids.shape[0]):
        new_tokens = generated_ids[i][len(inputs.input_ids[i]):]
        output_ids.append(new_tokens)

    content = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    content = [c.strip() for c in content]

    return content


For formatting into the prediction file and doing cleanup on the llm's responses

In [ ]:
%pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



#gets all the tails for a given entity and relation into one entry
def pred_cleanup(triples):
    predictions = []
    byEnt = triples.groupby("entity")

    #sort by relation
    for entg in byEnt:
        byRel = entg[1].groupby("relation")
        for rel in byRel:
            #print(rel[1]['object'].tolist())

            tails = rel[1]['object'].tolist()
            relation = rel[1]['relation'].tolist()[0]
            entity = rel[1]['entity'].tolist()[0]

            fixedtails = []

            #if it's empty, get into correct empty format
            if tails == ['none'] or tails == ["no"]:
                tails = []


            #if it's one of these two it should be floats
            elif relation == 'hasArea' or relation == 'hasCapacity':
                for t in tails:
                  
                  if isinstance(t, list):
                    print(t)
                    if t == []:
                        continue
                    else:
                        t = t[0]

                  print(t)
                  t = str(t)
                  if t == "0":
                    continue

                  t = t.split("\n")[0].replace(",", "")
                  try:
                    num = float(str(t))
                       # fixedtails.append(num)
                  except:
                    r = re.findall(r'-?\d+\.?\d*', t)
                    try:
                        num = float(r)
                    except:
                        num = llm_cleanup(t, entity, relation)#llmnumsplz(t)
                  fixedtails.append(num)
                tails = fixedtails

            #otherwise it should all be strings
            else:
                for t in tails:
                     if t != 'none' and str(t) != "0":
                        fixedtails.append(str(t))
                tails = fixedtails

            predictions.append({
        "SubjectEntity": rel[1]['entity'].tolist()[0],
        "Relation": relation,
        "ObjectEntities": tails
       # "ObjectEntitiesID": ["Q790", "Q717", "Q30", "Q183"]
    })

    print(predictions)
    return predictions

#makes sure format is correct for predictions file
def format_preds(triples, file):
    triples = pd.DataFrame(triples, columns = ["entity", "relation", 'object'])

    predictions = pred_cleanup(triples)
    predictions_ids = []

    for p in predictions:
        olist = p['ObjectEntities']

        if p['Relation'] == 'hasArea' or p['Relation'] == 'hasCapacity':
            p["ObjectEntitiesID"] = olist #for the numeric stuff there's no id, just the number

        else:
            if len(olist) > 0:
                ids = get_id(olist)
            else:
                ids = []

            p["ObjectEntitiesID"] = ids

        predictions_ids.append(p)

    fp = "data/"+ file + ".jsonl"

    with open(fp, "w") as f:
        for pred in predictions_ids:
            f.write(json.dumps(pred) + "\n")


def get_id(entityList):

    id_prompt = """Q: Wellington Island A: Q1048284 Q: DeGol Field in Pennsylvania A: Q100075 Q: {entity} A:"""

    ids = []
    for entity in entityList:
        c = call_llm(id_prompt.format(entity=entity), "temp_sampling")
        print(c)
        #c = c.split("Q:")[0].strip()
        c = c[0].split("Q:")[0].strip()
        try:
            c = c.split(" ")[0]
        except:
            print("ok so should just be one?")

        ids.append(c)

    if len(ids) != len(entityList):
        print("wrong nums")
    else:
        return ids


def get_reltype(rel):
    resType = ""
    if rel == "hasArea" or rel == "hasCapacity":
        resType = "quantities and numbers"
    elif rel == "companyTradesAtStockExchange":
        resType = "stock exchanges"
    elif rel == "awardWonBy":
        resType = "names of people"
    else:
        resType = "locations"
    return resType




def llm_cleanup(llmResponse, entity, rel):


    
    prompt = """
You will be given a text.
Your task is to extract any {type} from the text.
Give your answer in a python formatted list.

Here are examples of how to answer:
Input: "The Netherlands is a country in Northern Europe, situated between the North. Denmark is a"
Output: ["Netherlands", "Denmark"]

Input: ", which is located in the city of Toronto. The stadium has a seating capacity of 16,000 people. It's a relatively new facility, opened in 2009. The arena is known for its excellent acoust"
Output: ["16,000", "2009"]

Provide answer as follows:

Answer:::
Output: (your response, as a list of values)

You MUST provide values for 'Output:' in your answer.

Now here is the text.
Input: {input}

Answer:::
Output: """
#assuming a list for batching, rel will still be right
    print(rel)
    
    if not isinstance(llmResponse, list):
        resType = get_reltype(rel)
        rx = regexver(llmResponse, rel)
        if rx:
            return rx

        res = call_llm(prompt.format(type=resType, input=llmResponse), "temp_sampling")

        if len(res) == 1:
            r = res[0].split("\n")[0].replace("[", "").replace("]", "")
            return r
        return res

    rxres = []
    tofix = []
    allprompts = []

    for i, (r, ent, rl) in enumerate(zip(llmResponse, entity, rel)):
        #print("IN REGEX REL LIST ", rl)
        if "Q:" in r:
            cleaned = r.split("Q:")[0]
        else:
            cleaned = r
        #print("regex given", cleaned, rl)
        rx = regexver(cleaned, rl)
        #print("REGEX ", rx)
        if rx and rx != None:
            if isinstance(rx, list) and len(rx) ==1:
                rx = rx[0]
            rxres.append((i, rx))
        else:
           # print("llm check")
            tofix.append(i)
            resType = get_reltype(rl)
            allprompts.append(prompt.format(type=resType, input=cleaned))

    if len(allprompts) !=0:
        llmres = call_llm(allprompts, "temp_sampling")
    else:
        llmres = []

    resdone = [None] * len(llmResponse)

    for i, result in rxres:
        resdone[i] = result

    for ii, llmi in enumerate(tofix):
        if ii < len(llmres):
            res = llmres[ii]

            if "\n" in res:
                res = res.split("\n")[0]

            if "[" in res and "]" in res:
                try:
                    res = ast.literal_eval(res)
                except:
                    res = str(res)

            resdone[llmi] = res

    print("cleaned ", resdone)
    return resdone

import re
import ast

def regexver(text, rel):
    if "Q:" in text:
        text = text.split("Q:")[0]

    if rel == "hasArea" or rel == "hasCapacity":
        pattern = r'\d+(?:,\d+)*(?:\.\d+)?'
        res = re.findall(pattern, text)
        if res:
          return res
        else:
          return None

    else:
        pattern = r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b'
        extrawords = ['okay', 'ok', "Okay", "(your response, as a list of values)", "Ok"]

        cleanedr = []
        res = re.findall(pattern, text)
        for r in res:
            words = r.split()
            if not all(word.lower() in stop_words or word in extrawords for word in words):
                cleanedr.append(r)
        if cleanedr:
          return cleanedr
        else:
          return None




Checks for multiple instances of a prediction and consolidates from the initial set (as 5-10 triples exist for each entity/relation pair at this point)

In [ ]:
from collections import Counter

def consensus(tripleList, minimum = 2, regex = False, handlesingles = False):
	#min is number of times it should show up to keep it
	multianswers = ["companyTradesAtStockExchange", "awardWonBy","countryBordersCountry"]
	#some relations only have a single answer, so only taking the one with the most hits
	goodTriples =[]
	tl = pd.DataFrame(tripleList, columns = ["entity", 'relation', 'object' ])
	byent = pred_cleanup(tl)

	for triple in byent:
		tocheck = triple["ObjectEntities"]
		#print(tocheck)
		prompt = """
You will be given a list.
Your task is to select items in the list appear {min} or more times.
Give your answer in the following format

Here are examples of how to answer:
Input: ["the eu", "EU", "European Union", "the european union", "France", "Italy", "Italia"]
Output: European Union # Italy
Input: [5000.0, 5000.0, 5000, 5000, 5000.0, 5000]
Output: 5000

Provide answer as follows:

Answer:::
Output: (your response, with the values that appear {min} or more times)

You MUST provide values for 'Output:' in your answer.
Now here is the list.
Input: {tails}

Answer:::
Output: """
		
		tocheck = [x for x in tocheck if x not in [0, 0.0, '0', '0.0']] #taking out the stuff that is just 0, bc always wrong
		#print(regex, handlesingles)
		if regex and handlesingles:
			nums = Counter(tocheck)
			if triple["Relation"] == "hasArea":
				print(nums)
				print(tocheck)

			if triple["Relation"] in multianswers:
				goodones = [item for item, count in nums.items() if count >= minimum]
				if goodones:
					for tail in goodones:
						goodTriples.append((triple['SubjectEntity'], triple['Relation'], tail))
				else: 
					goodTriples.append((triple['SubjectEntity'], triple['Relation'], "none"))
			else:
				try:
					bestval = max(nums, key=nums.get)
					if bestval:
						goodTriples.append((triple['SubjectEntity'], triple['Relation'], bestval))
				except:
					goodTriples.append((triple['SubjectEntity'], triple['Relation'], "none"))

		elif regex:
			#if ignoring that some of the answers should only be a single value
			nums = Counter(tocheck)
			goodones = [item for item, count in nums.items() if count >= minimum]
			if goodones:
				for tail in goodones:
					goodTriples.append((triple['SubjectEntity'], triple['Relation'], tail))
			else: 
				goodTriples.append((triple['SubjectEntity'], triple['Relation'], "none"))

		else:

			res = call_llm(prompt.format(tails = tocheck, min = min), "temp_sampling", True)
			print(res)
			res = res[0]
			if type(res) == list:
				for r in res:
					r = r.strip().replace("\\", "").replace('"', "")
					if '#' in r:
						r = r.split("#")
						for rm in r:
							goodTriples.append((triple['SubjectEntity'], triple['Relation'], rm))
					else:
						goodTriples.append((triple['SubjectEntity'], triple['Relation'], r))
			else:
				r = res
				r = r.strip().replace("\\", "").replace('"', "")
				if '#' in r:
					r = r.split("#")
					for rm in r:
						goodTriples.append((triple['SubjectEntity'], triple['Relation'], rm))
				else:
					goodTriples.append((triple['SubjectEntity'], triple['Relation'], r))

	return goodTriples




Cleaning the llm response for numbers specifically, and the judge for the re-run bit, which checks each triple 3x in case of wild responses

In [ ]:
def llmnumsplz(input):

    prompt = """
You will be given a text.
Your task is to extract any number from the text.
Give your answer in numbers without any commas or unit measurements.

Here are examples of how to answer:
Input: ", which is located in the city of Toronto. The stadium has a seating capacity of 16,000 people. It's a relatively new facility, opened in 2009. The arena is known for its excellent acoust"
Output: 16000

Input: "I think the answer is 75"
Output: 75

Input: "20 thousand km sq"
Output: 20000

Provide answer as follows:

Answer:::
Output: (your response, a number)

You MUST provide values for 'Output:' in your answer.

Now here is the text.
Input: {input}

Answer:::
Output: """


    if type(input) == list: #so if s number agreement check
        plist = []
        for x in input:
            plist.append(prompt.format(input = x))

        print(plist)


        res = call_llm(plist[0], "temp_sampling")
        print(res)
        cleanlist = []
        for r in res:
            r = r.replace(",", "")
            try:
                if "\n" in r:
                    r = r.split("\n")[0]

                f = float(r)
                cleanlist.append(r)
            except ValueError:
                print("try again ", r)
                cleanlist.append(r)
        return cleanlist

    else: #just clean out the number
        print(prompt.format(input = input))
        res = call_llm(prompt.format(input = input), "temp_sampling")


        if "\n" in res:
            res = res.split("\n")[0]
        try:
            if type(res) == list:
                res = res[0]
            f = float(str(res))
            return f
        except ValueError:
            print("try again ", res)
            return 0

def minijudge(triple, temps = False, single = False):
    #after rerun stuff before formatting so list of tuples to list of tuples
    #scores = []

        #adapted from https://huggingface.co/learn/cookbook/llm_judge
    prompt = """
You will be given a knowledge graph triple.
Your task is to provide a 'total rating' scoring how confident you are that the triple is accurate.
Give your answer on a scale of 0 to 100, where 0 means that you believe the triple is inaccurate, and 100 means that the triple is absolutely accurate.

Here is the scale you should use to build your answer:
0: The triple is terrible: the tail is irrelevant or incorrect given the relation
25: The triple is probably not correct: the tail might make sense given the relation but the information is wrong
50: The triple is probably correct: the tail makes sense given the relation and the information might be right
100: The triple is excellent: the tail makes sense given the relation and the information is correct

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as a number between 0 and 100)

You MUST provide values for 'Total rating:' in your answer.

Now here is the triple.
{entity} {rel} {tail}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Feedback:::
Total rating: """

    prompts = []
    for t in triple:
        prompts.append(prompt.format(entity = t[0], rel = t[1], tail = t[2]))

    if single:
        #print("minijudge vals in", triple)
        #not doing the 3 check, only for re-run to reduce computation time
        try:
          res = call_llm(prompts, "temp_sampling") #using default temp
        except:
           res = call_llm(prompts, "temp_sampling") #if it didn't work the firts time, just have it try again
        cleanedres = []
        for r in res:
          if not re.findall(r'-?\d+\.?\d*', str(r)):
            r = 50
          else:
            r = str(r)
            r = r.split("\n")[0]
            r = r.replace(",", "")

            if r == []:
              r = "50"

            if r == '':
              r = "50"

            try:
                r = float(r)
            except:
                r = llm_cleanup(r, "", "hasArea")#llmnumsplz(r)
                try:
                  float(r)
                except:
                  r = 50

            cleanedres.append(r)

        verdict = []
       # print("cleanedres minijudge ", cleanedres)
        for r in cleanedres:
            if type(r) == list and r != []:
                if type(r[0]) == list:
                    r = [float(item) for sublist in r for item in sublist]
            #r = [x if isinstance(x, (int, float)) else 50.0 for x in r]

            
            if r > 50: #if average over 3 checks is above threshold, keep triple
                verdict.append("yes")
            else:
                verdict.append("no")
       # print(verdict)
        return verdict
    
    if temps:
      t1 = .2
      t2 = .4
      t3 = .6
    else:
      t1 = t2 = t3 = .15
    r1 = call_llm(prompts, "temp_sampling", t = t1)
    r2 = call_llm(prompts, "temp_sampling" , t = t2)
    r3 = call_llm(prompts, "temp_sampling", t = t3)
    allres = zip(r1,r2,r3) #list of lists now?

    allclean = []

    for res in allres:
        cleanedres = []
        for r in res:
          if not re.findall(r'-?\d+\.?\d*', str(r)):
            r = 50
          else:
            r = str(r)
            r = r.split("\n")[0]
            r = r.replace(",", "")

            if r == []:
              r = "50"

            if r == '':
              r = "50"

            try:
                r = float(r)
            except:
                r = llm_cleanup(r, "", "hasCapacity")
                try:
                  float(r)
                except:
                  r = 50

            cleanedres.append(r)

        allclean.append(cleanedres)#should maintain order, since zip will have all of each in a list, then cleaning each list?

    verdict = []
    for r in allclean:
        if type(r) == list and r != []:
          if type(r[0]) == list:
            r = [float(item) for sublist in r for item in sublist]
        r = [x if isinstance(x, (int, float)) else 50.0 for x in r]

        #print(r)
        if len(r) == 0:
          verdict.append("no")
        elif sum(r)/len(r) > 50: #if average over 3 checks is above threshold, keep triple
            verdict.append("yes")
        else:
            verdict.append("no")
    return verdict


One of the validation/checking versions where the candidate triple is converted into a natual language and then compared with the original triple, and if similar enough it's retained

In [ ]:
def translate_check(tripleList):
    #basically have an llm translate the triple into natural language, then do another call asking if they're the same if yes keep
    goodTriples = []
    prompt = """
        You will be given a triple and a sentence.
Your task is to compare the triple and the sentence and determine if they have the same meaning, particularly whether the tail of the triple is consistent.
Give your answer as yes or no with no other text
Provide answer as follows:
Here are examples of how to answer:
Input: T:(Ilha da Queimada Grande, hasArea, 0.43) S: The area of Ilha da Queimada Grande is .43.
Output: yes
Input: T:(Ilha da Queimada Grande, hasArea, 0.43) S: The Ilha da Queimada Grande has an area of .43.
Output: yes
Input: T:(Ilha da Queimada Grande, hasArea, 0.43) S: The Ilha da Queimada Grande area is .7.
Output: no

Provide answer as follows:
Answer:::
Output: (your response, yes or no)

You MUST provide values for 'Output:' in your answer.
Now here is the triple.
Input: T:{triple} S:{sentence}

Answer:::
Output: """

    promptsentence = """
You will be given a triple.
Your task is to convert the triple to a natural language sentence.
Give your answer and no other text
Provide answer as follows:

Answer:::
Output: (your response)

You MUST provide values for 'Output:' in your answer.
Now here is the triple and sentence.
Input: {tails}

Answer:::
Output: """

    i = 0
    withprompt = {}
    for t in tripleList:
      withprompt[i] = {"triple": t, "promptsentence": promptsentence.format(tails = t) }

      i = i+1



    keys = list(withprompt.keys())
    for i in range(0, len(keys), 5):
        k = keys[i:i + 5]
        cp = [withprompt[key]["promptsentence"] for key in k]

        res = call_llm(cp, "temp_sampling")

        for key, response in zip(k, res):
            withprompt[key]["sentence"] = response
        #makes all the sentences for the triples
    print("sentences done")

    for key in withprompt:
      withprompt[key]["prompt"] = prompt.format(triple = withprompt[key]["triple"], sentence = withprompt[key]["sentence"])


    for i in range(0, len(keys), 5):
      k = keys[i:i + 5]
      cp = [withprompt[key]["prompt"] for key in k]

      res = call_llm(cp, "temp_sampling")

      for key, response in zip(k, res):
        withprompt[key]["res"] = response



    for key in withprompt:
      r = withprompt[key]["res"]
      if 'yes' in r:
        goodTriples.append(withprompt[key]["triple"])
    print("translate finished")
    return goodTriples







Another validation check which uses the judge from the re-run to check whether the triple is 'good' or not

In [ ]:
def confidence_new(triples, alt = False):
  goodTriples = []
  check = {}
  i = 0
  for t in triples:
    check[i] = {"triple": t, "good?": ""}
    i = i + 1


  keys = list(check.keys())

  for i in range(0, len(keys), 5):
            k = keys[i:i + 5]
            cp = [check[key]["triple"] for key in k]

            res = minijudge(cp, alt)

            for key, response in zip(k, res):
                check[key]["good?"] = response

  for key, item_dict in check.items():
            if "yes" in item_dict["good?"]:
                goodTriples.append(item_dict["triple"])
  return goodTriples



Handles the selection of custom examples from the validation set for each entity/relation pair

In [ ]:
#to get the examples prepped for inital generation for test

def custom_examples(entity, rel, options):
    #get from the validation set (since using the training set for the data)

  relall = options.get_group(rel)


  flist = [f"{i+1}. {r['SubjectEntity']} {r['Relation']} {r['ObjectEntities'][0]}" for i, (_, r) in enumerate(relall.iterrows())]
  print(flist)

  prompt = """
You will be given a list of knowledge graph triples and a target triple.
Your task is to provide the 3 triples from the given list that are most similar to the target.
Give your answer as the 3 numbers representing the most similar triples to the target triple.

Provide your answer as follows:

Answer:::
3 Similar Triples: (your answer, the numbers associated with the most similar 3 triples to the candidate)

You MUST provide values for '3 Similar Triples:' in your answer.

Now here is the list of triples:
{list}

And here is the target triple:
{entity} {rel}
Provide your answer. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Answer:::
3 Similar Triples: """

  res = call_llm(prompt.format(list = flist, entity = entity, rel = rel), "temp_sampling")
  print(res)
  res = res[0]
  try:
    res = res.strip().split(",")
  except:
     res = res.split(" ")

  #res = ['1 2 3']
  if len(res) < 3:
     try:
        res = res[0].split(" ")
     except:
        res = res.split(" ")


  examples = []
  for v in res:
     examples.append(relall.iloc[int(v)-1])
     print(flist[int(v)-1])
     print(relall.iloc[int(v)-1])

  print(res)
  cleaned = []
  for r in examples:
      textver = ""
      t = ""
          #

      try:
          t = t + "Q: " + r['SubjectEntity'] + " # " + r['Relation'] + "\n" + "A: " + r["ObjectEntities"][0]
          if len(r["ObjectEntities"]) > 1:
            for vs in r["ObjectEntities"][1:]:
                t = t + " # " + vs
      except:
          t = t + "Q: " + r['SubjectEntity'] + " # " + r['Relation'] + "\n" + "A: none"
      textver = textver + t + "\n"
      cleaned.append(textver)
  print(cleaned)
  return cleaned



def fix(x):
    if isinstance(x, list) and len(x) == 0:
        return ["none"]
    return x



def get_vals():
  with open('./data/val.jsonl', 'r') as file:
    vallist = list(file)

  examples = []
  for v in vallist:
      r = json.loads(v)
      examples.append((r['SubjectEntity'], r['Relation'], r["ObjectEntities"]))

  exampledf = pd.DataFrame(examples, columns = ['SubjectEntity', 'Relation', "ObjectEntities" ] )
  exampledf["ObjectEntities" ] = exampledf["ObjectEntities" ].apply(fix)

  rd = exampledf.groupby('Relation')#.apply(lambda x: x.values.tolist()).to_dict()
  #double check this is actually getting all the data
  return rd



#running custom examples for training set, using validation to get the examples
import pandas as pd


import json

with open('./data/test.jsonl', 'r') as json_file:
    json_list = list(json_file)

data = []
for json_str in json_list:
    r = json.loads(json_str)
    data.append((r['SubjectEntity'], r['Relation'], r["SubjectEntityID"]))
print(len(data))
print(data)


options = get_vals()


triples = data
wexamples = {}
for triple in triples:
    entity = triple[0]
    rel = triple[1]

    res = custom_examples(entity, rel, options)

    wexamples[entity + "," + rel] = res

    print(wexamples)


df = pd.DataFrame(wexamples, columns=["triple", "examples"])
df.to_csv("data/testexamples.tsv", sep = "\t")
print(len(data))
print(len(wexamples))
#so will be a dict with "entity,rel" as the key and then the list of 3 examples afterward


Loading in test data

In [ ]:
#cuing up test data

with open('./data/test.jsonl', 'r') as json_file:
    json_list = list(json_file)

data = []
for json_str in json_list:
    r = json.loads(json_str)
    data.append((r['SubjectEntity'], r['Relation'], r["SubjectEntityID"]))
print(len(data))
print(data)

Run on the test data set which is used in the challenge

In [ ]:
triples = initial_triple(data, "temp_sampling")
df = pd.DataFrame(triples, columns = ["entity", "rel", "tail"])
df.to_csv("testinital.tsv", sep = "\t")

rerun = faster_check(triples, "temp_sampling")
df = pd.DataFrame(rerun, columns = ["entity", "rel", "tail"])
df.to_csv("testrerun.tsv", sep = "\t")

cs = consensus(rerun)
checked = confidence_new(cs, temps = True)

format_preds(checked, "testpredictions")


#below is what was submitted


csregexsingle = consensus(rerun, regex = True, handlesingles = True)
checked = confidence_new(csregexsingle,  True)

format_preds(csregexsingle, "predictionsnojudge")
format_preds(checked, "predictionsjudged")


In [ ]:
# the numbers are supposes to be strings first??????


import json
def fixnums(file, newfile):

    with open(file, 'r') as infile, open(newfile, 'w') as outfile:
        
        for line in infile:
            data = json.loads(line.strip())
            
            # Convert each element in the list to string
            data["ObjectEntities"] = [str(item) for item in data["ObjectEntities"]]
            
            outfile.write(json.dumps(data) + '\n')
fixnums("regexwjudge.jsonl", "fixedregexwjudge.jsonl")
fixnums("originalconsensusjudge.jsonl", "fixedogconsensus.jsonl")
fixnums("llmconsensus.jsonl", "fixedllmconsensus.jsonl")

In [ ]:
#The generation for the submitted version - used only good triples from re-run and regex consensus
# fixedtheorywjudge was the submitted file
df = pd.read_csv("data/4.tsv", sep = "\t")
triples = list(zip(df['entity'], df['rel'], df['tail']))

cs = consensus(triples, regex =True, handlesingles=True)
format_preds(cs, "testingtheory")

confidence = confidence_new(cs, True)
format_preds(confidence, "theorycheckwjudge")

fixnums("data/testingtheory.jsonl", "fixedtheorytest.jsonl")
fixnums("data/theorycheckwjudge.jsonl", "fixedtheorywjudge.jsonl")